In [150]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from datetime import timedelta
import math 

In [151]:

# Cargar archivo Excel
file_path = "generic_input_case.xlsx"

# Leer cada hoja
df_horizonte = pd.read_excel(file_path, sheet_name="HORIZONTE")
df_bd_up = pd.read_excel(file_path, sheet_name="BD_UP")
df_frota = pd.read_excel(file_path, sheet_name="FROTA")
df_grua = pd.read_excel(file_path, sheet_name="GRUA")
df_fabrica = pd.read_excel(file_path, sheet_name="FABRICA")
df_rota = pd.read_excel(file_path, sheet_name="ROTA")



c:\Users\wilma\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [152]:
def constructive (df_horizonte, df_bd_up, df_frota, df_grua, df_fabrica, df_rota):
    #Data of the problem (size of elemtns to consider)
    ups=len(df_bd_up)
    transportadores=df_frota["TRANSPORTADOR"]
    l_transportadores=len(df_frota)
    dias=len(df_horizonte)

    #Construction of dictionaries with information needed
        #Fazendas covered by each transporter
    fazendas_t={}
    for t in df_frota["TRANSPORTADOR"]:
        faz=[]
        for idx, ruta in df_rota.iterrows():
            if ruta["TRANSPORTADOR"]==t and ruta["Fazenda"] not in faz:
                faz.append(ruta["Fazenda"])
        fazendas_t[t]=faz
        #UP in each Fazenda
    up_faz={}
    fazendas=list(df_bd_up["FAZENDA"].unique())
    for f in fazendas:
        ups_i=[]
        for idx, upi in df_bd_up.iterrows():
            if upi["FAZENDA"]==f and upi["UP"] not in ups_i:
                ups_i.append(upi["UP"])
        up_faz[f]=ups_i

        #transporters that cover each UP
    transp_up={}
    for up in df_bd_up["UP"]:
        up_i=[]
        for t in df_frota["TRANSPORTADOR"]:
            for idx, ruta in df_rota.iterrows():
                if ruta["TRANSPORTADOR"]==t and ruta["ORIGEM"]==up and ruta["TRANSPORTADOR"] not in up_i:
                    up_i.append(ruta["TRANSPORTADOR"])
        transp_up[up]=up_i

    #Solution will be constructed based on hierarchical rules (most volume that complies with the RSP first)
    df_up_sort=df_bd_up.sort_values(by=["VOLUME"],ascending=False).reset_index(drop=True)
    # print(df_up_sort)
    rsp_min=df_fabrica.iloc[0]["RSP_MIN"]
    rsp_max=df_fabrica.iloc[0]["RSP_MAX"]
    d_min=df_fabrica.iloc[0]["DEMANDA_MIN"]
    d_max=df_fabrica.iloc[0]["DEMANDA_MAX"]
    #First to assign those UP that the RSP is within the range
    best_i=df_up_sort[(df_up_sort["RSP"]>=rsp_min)& (df_up_sort["RSP"]<=rsp_max)].index.to_list()
    
    a=df_up_sort.iloc[best_i]
    o_best_i=list(set(df_up_sort.index.to_list()) - set(best_i))
    b=df_up_sort.iloc[o_best_i]
    #sorted dataframe according to new rules
    df_sorted=pd.concat([a,b],axis=0,ignore_index=True)

    x=np.zeros([dias,ups,l_transportadores],dtype=int)
    d_array=np.zeros([dias,ups,l_transportadores])


    d_list=[i for i in range(dias)]
    ups_list={up:i for i,up in enumerate (df_sorted["UP"].to_list())}
    t_list={t:i for i,t in enumerate (df_frota["TRANSPORTADOR"].to_list())}
    ##LIMIT DOWNNNNN df_sorted.iloc[:1]
    vol_available={up_i["UP"]:up_i["VOLUME"] for _,up_i in df_sorted.iterrows()}

    # print(vol_available)

    max_up_t={t["TRANSPORTADOR"]:t["QTD_GRUAS"] for  _,t in df_grua.iterrows()}
    perc_t={t["TRANSPORTADOR"]:t["PORCENTAGEM_VEICULOS_MIN"] for  _,t in df_grua.iterrows()}    
    max_t={t["TRANSPORTADOR"]:t["FROTA_MAX"] for  _,t in df_frota.iterrows()}
    min_t={t["TRANSPORTADOR"]:t["FROTA_MIN"] for  _,t in df_frota.iterrows()}

    d=0
    while d <= 8:
        
        print(f"Day: {d}")
        cars_x_transp={t:max_t[t] for t in transportadores} 
        print(f'Attended demand: {np.sum(d_array[d,:,:])} units')
        if np.sum(d_array[d,:,:])>d_min and np.sum(d_array[d,:,:])<d_max:
            print(f"¡El día {d} cuenta con la demanda satisfecha!")
            d+=1
        
        else:
            for up in vol_available:
                v_up=vol_available[up]
                if v_up>0:
                    transp_i=transp_up[up]
                    caixa_carga={t:0 for t in transp_i}

                    tempo_c={t:0 for t in transp_i}
                    
                    for t_i in transp_i:
                        caixa_carga[t_i]+=df_rota[(df_rota["ORIGEM"]==up) & (df_rota["TRANSPORTADOR"]==t_i)]["CAIXA_CARGA"].to_list()[0]
                        tempo_c[t_i]+=df_rota[(df_rota["ORIGEM"]==up) & (df_rota["TRANSPORTADOR"]==t_i)]["TEMPO_CICLO"].to_list()[0]

                    max_truck_d={t:caixa_carga[t]*tempo_c[t] for t in transp_i}
                    days_to_cover_max={t:[0,0] for t in transp_i}

                    for t_i in transp_i:
                        ratio=v_up/(max_t[t_i]*max_truck_d[t_i])
                        r_up=math.ceil(ratio)
                        diff=r_up-ratio
                        days_to_cover_max[t_i][0]=r_up
                        days_to_cover_max[t_i][1]=diff

                    ass_order=sorted(days_to_cover_max, key=lambda kv: kv[2])
                    if cars_x_transp[ass_order[0]]<=0:
                        ass_order.pop(0)
                        if len(ass_order)<1:
                            d+=1
                            continue

                    if min(np.array(list(days_to_cover_max.values()))[:,0])+d<=max(d_list) and cars_x_transp[ass_order[0]]>0 and vol_available[up]>0:
                        t_i=ass_order[0]
                        t_x_idx=t_list[t_i]
                        up_x_idx=ups_list[up]
                        day_ass=days_to_cover_max[t_i][0]
                        cars=max_t[t_i]
                        ass_veh=min(cars_x_transp[t_i], cars)
                        x[d:d+day_ass,up_x_idx,t_x_idx]+=ass_veh
                        vol_available[up]=0
                        cars_x_transp[t_i]-=ass_veh
                        ratio=v_up/(max_t[t_i]*max_truck_d[t_i])

                        for d_i in range(day_ass):
                            # print("loop",d_i)
                            if d_i<ratio-1:
                                d_array[d+d_i,up_x_idx,t_x_idx]+=float(ass_veh)*float(max_truck_d[t_i])
                            else:
                                d_array[d+d_i,up_x_idx,t_x_idx]+=float(ass_veh)*float(max_truck_d[t_i])*(1-(day_ass-ratio))
                        if np.sum(d_array[d,:,:])>d_min and np.sum(d_array[d,:,:])<d_max:
                            d+=1
                else:
                    d+=1
                    continue #next up to analyze
            
    
    # print(t_list)
    # print(d_array[0:8,:,:])

constructive(df_horizonte,df_bd_up,df_frota,df_grua,df_fabrica,df_rota)

Day: 0
Attended demand: 0.0 units


In [159]:
# 📚 Constructive Heuristic for Prescriptive Analytics Competition
# ---
# ✅ Heurística constructiva refinada para cumplir toda la normativa:
#   • Prioridad por volumen y RSP.
#   • Tamaño de flota diario (min/max).
#   • Límite de grúas (UP simultáneas).
#   • Porcentaje mínimo de vehículos por UP.

import pandas as pd
import numpy as np

# 📂 Carga de datos
data = pd.ExcelFile("generic_input_case.xlsx")
horizonte = data.parse("HORIZONTE")
bd_up_orig = data.parse("BD_UP")
frota = data.parse("FROTA")
fabrica = data.parse("FABRICA")
rota = data.parse("ROTA")
grua = data.parse("GRUA")

# ⚙️ Preparación de datos
# Asociar UPs a transportadores y capacidades
bd_up = bd_up_orig.merge(rota, left_on='UP', right_on='ORIGEM', how='left')
bd_up['VOLUME_RESTANTE'] = bd_up['VOLUME']
bd_up['VOLUME_POR_CAMION'] = bd_up['CAIXA_CARGA'] * bd_up['TEMPO_CICLO']

# Demandas diarias de fábrica
daily = fabrica.set_index('DIA')[['DEMANDA_MIN','DEMANDA_MAX','RSP_MIN','RSP_MAX']].copy()
daily['ATENDIDO'] = 0

# Flota constante por transportador
fleet = frota.set_index('TRANSPORTADOR')[['FROTA_MIN','FROTA_MAX']]
# Gruas y porcentaje mínimo
cranes = grua.set_index('TRANSPORTADOR')[['QTD_GRUAS','PORCENTAGEM_VEICULOS_MIN']]

assignments = []

# Iterar cada día
for day in sorted(daily.index):
    print(f"--- Día {day} ---")
    # Inicializar estado de cada transportador
    state = {}
    for t, (fmin, fmax) in fleet.iterrows():
        if t not in cranes.index:
            continue
        qtd_gruas, pct_min = cranes.loc[t]
        state[t] = {
            'veh_asignados': 0,
            'ups': set(),
            'fmin': int(fmin),
            'fmax': int(fmax),
            'qtd_gruas': int(qtd_gruas),
            'pct_min': pct_min/100
        }

    # Candidatos iniciales: todas las UPs con volumen pendiente
    cand = bd_up[bd_up['VOLUME_RESTANTE'] > 0].copy()

    # Mientras exista demanda y candidatos
    while daily.at[day,'ATENDIDO'] < daily.at[day,'DEMANDA_MAX'] and not cand.empty:
        # Priorizar UP según RSP y volumen
        rsp_min, rsp_max = daily.at[day,'RSP_MIN'], daily.at[day,'RSP_MAX']
        sel = cand[cand['RSP'].between(rsp_min, rsp_max)].nlargest(1,'VOLUME_RESTANTE')
        if sel.empty:
            sel = cand[cand['RSP'].between(rsp_min, rsp_max)].nsmallest(1,'VOLUME_RESTANTE')
        if sel.empty:
            sel = cand.nlargest(1,'VOLUME_RESTANTE')
        if sel.empty:
            sel = cand.nsmallest(1,'VOLUME_RESTANTE')

        up = sel.iloc[0]
        t = up['TRANSPORTADOR']
        # Si transportador no existe o no tiene flota, eliminar candidato
        if t not in state or state[t]['fmax'] == 0:
            print(f"Omitido UP {up['UP']} porque transportador {t} no disponible")
            cand = cand[cand['UP'] != up['UP']]
            continue

        st = state[t]
        # Restricción de grúas
        if len(st['ups']) >= st['qtd_gruas']:
            print(f"{t} ya opera {st['qtd_gruas']} UPs — límite de grúas. Omitiendo {up['UP']}")
            cand = cand[cand['UP'] != up['UP']]
            continue
        # Restricción de flota
        max_flot = st['fmax'] - st['veh_asignados']
        if max_flot <= 0:
            print(f"{t} sin flota restante. Omitiendo {up['UP']}")
            cand = cand[cand['UP'] != up['UP']]
            continue

        # Calcular número de camiones
        vol_disp = daily.at[day,'DEMANDA_MAX'] - daily.at[day,'ATENDIDO']
        vol_up = up['VOLUME_RESTANTE']
        vol_cam = up['VOLUME_POR_CAMION']
        trucks = min(int(np.floor(min(vol_disp,vol_up)/vol_cam)), max_flot)
        trucks = max(trucks,1)
        min_trucks = int(np.ceil(st['pct_min'] * st['fmin']))
        if trucks < min_trucks:
            trucks = min_trucks
            if trucks > max_flot:
                print(f"No factible {min_trucks} camiones para {up['UP']} por flota")
                cand = cand[cand['UP'] != up['UP']]
                continue
        vol_ass = min(trucks * vol_cam, vol_up, vol_disp)
        print(f"{t}: {trucks} camiones -> UP {up['UP']} vol {vol_ass}")

        # Registrar asignación
        assignments.append({
            'UP': up['UP'],
            'FAZENDA': up['FAZENDA'],
            'TRANSPORTADOR': t,
            'DIA': day,
            'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
            'DB': up['DB'],
            'RSP': up['RSP'],
            'QTD_VEICULOS': trucks,
            'VOLUME': vol_ass
        })
        # Actualizar
        bd_up.loc[(bd_up['UP']==up['UP']) & (bd_up['TRANSPORTADOR']==t),'VOLUME_RESTANTE'] -= vol_ass
        daily.at[day,'ATENDIDO'] += vol_ass
        st['veh_asignados'] += trucks
        st['ups'].add(up['UP'])
        # Eliminar candidato si se agotó
        if bd_up.loc[(bd_up['UP']==up['UP']) & (bd_up['TRANSPORTADOR']==t),'VOLUME_RESTANTE'].iloc[0] <= 0:
            cand = cand[cand['UP'] != up['UP']]

    print(f"Total atendido día {day}: {daily.at[day,'ATENDIDO']}\n")

# Exportar resultado
output_df = pd.DataFrame(assignments)
output_df.to_excel("output_solution.xlsx", index=False)
print(f"Asignaciones totales: {len(assignments)}")
print("Solución guardada en output_solution.xlsx")


c:\Users\wilma\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:122: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4111.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype fir

--- Día 1 ---
Rampazo: 21 camiones -> UP S5AW13 vol 2079.0
Rampazo sin flota restante. Omitiendo S5AW13
Rampazo sin flota restante. Omitiendo S5AK10
Rampazo sin flota restante. Omitiendo S5AW10
Tover: 14 camiones -> UP S6BX01 vol 2032.8000000000002
Tover sin flota restante. Omitiendo S6BX01
Rampazo sin flota restante. Omitiendo S5AK05
Rampazo sin flota restante. Omitiendo S5AW05
Rampazo sin flota restante. Omitiendo S5AW09
Rampazo sin flota restante. Omitiendo S5AK09
Rampazo sin flota restante. Omitiendo S5AW14
Rampazo sin flota restante. Omitiendo S5AK08
Tover sin flota restante. Omitiendo S6BG12
Rampazo sin flota restante. Omitiendo S3AX01
Tover sin flota restante. Omitiendo S6C335
Tover sin flota restante. Omitiendo S6C334
Rampazo sin flota restante. Omitiendo S3AX06
Tover sin flota restante. Omitiendo S6C297
Tover sin flota restante. Omitiendo S6C421
Tover sin flota restante. Omitiendo S6BG13
Rampazo sin flota restante. Omitiendo S3AX02
Tover sin flota restante. Omitiendo S6C298
Ra

C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) in

Rampazo sin flota restante. Omitiendo S5AK09
Rampazo sin flota restante. Omitiendo S5AW14
Rampazo sin flota restante. Omitiendo S5AK08
Rampazo sin flota restante. Omitiendo S3AX01
Tover: 14 camiones -> UP S6BG12 vol 2310.0
Tover sin flota restante. Omitiendo S6BG12
Tover sin flota restante. Omitiendo S6C335
Tover sin flota restante. Omitiendo S6C334
Rampazo sin flota restante. Omitiendo S3AX06
Tover sin flota restante. Omitiendo S6C297
Tover sin flota restante. Omitiendo S6C421
Tover sin flota restante. Omitiendo S6BG13
Rampazo sin flota restante. Omitiendo S3AX02
Tover sin flota restante. Omitiendo S6C298
Rampazo sin flota restante. Omitiendo S3AX03
Tover sin flota restante. Omitiendo S6BG08
Rampazo sin flota restante. Omitiendo S3AX04
Tover sin flota restante. Omitiendo S6BG11
Tover sin flota restante. Omitiendo S6BG10
Tover sin flota restante. Omitiendo S6BG14
Total atendido día 8: 6923.4

--- Día 9 ---
Rampazo: 21 camiones -> UP S5AW13 vol 2079.0
Rampazo sin flota restante. Omitien

C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) in

...

C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) in

Rampazo sin flota restante. Omitiendo S5AK09
Rampazo sin flota restante. Omitiendo S5AW14
Rampazo sin flota restante. Omitiendo S5AK08
Rampazo sin flota restante. Omitiendo S3AX01
Rampazo sin flota restante. Omitiendo S3AX06
Rampazo sin flota restante. Omitiendo S3AX02
Rampazo sin flota restante. Omitiendo S3AX03
Tover: 14 camiones -> UP S6C334 vol 1940.3999999999999
Tover sin flota restante. Omitiendo S6C334
Tover sin flota restante. Omitiendo S6BG13
Tover sin flota restante. Omitiendo S6BG08
Tover sin flota restante. Omitiendo S6C298
Tover sin flota restante. Omitiendo S6BG12
Rampazo sin flota restante. Omitiendo S3AX04
Tover sin flota restante. Omitiendo S6BG11
Tover sin flota restante. Omitiendo S6BG10
Tover sin flota restante. Omitiendo S6C297
Tover sin flota restante. Omitiendo S6BG14
Tover sin flota restante. Omitiendo S6C335
Total atendido día 24: 6692.4

--- Día 25 ---
Pastori: 27 camiones -> UP S5AW09 vol 2673.0
Pastori sin flota restante. Omitiendo S5AW09
Rampazo: 21 camione

C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) in

Tover sin flota restante. Omitiendo S6BG12
Rampazo sin flota restante. Omitiendo S3AX04
Tover sin flota restante. Omitiendo S6BG11
Tover sin flota restante. Omitiendo S6BG10
Tover sin flota restante. Omitiendo S6C297
Tover sin flota restante. Omitiendo S6BG14
Tover sin flota restante. Omitiendo S6C335
Tover sin flota restante. Omitiendo S6C334
Total atendido día 27: 6553.799999999999

--- Día 28 ---
Rampazo: 21 camiones -> UP S5AK05 vol 1940.3999999999999
Rampazo sin flota restante. Omitiendo S5AK05
Rampazo sin flota restante. Omitiendo S5AW13
Pastori: 16 camiones -> UP S5AK10 vol 1478.3999999999999
Pastori: 11 camiones -> UP S5AK10 vol 1016.3999999999999
Rampazo sin flota restante. Omitiendo S5AW09
Rampazo sin flota restante. Omitiendo S5AW05
Rampazo sin flota restante. Omitiendo S5AW10
Rampazo sin flota restante. Omitiendo S5AK09
Rampazo sin flota restante. Omitiendo S5AW14
Rampazo sin flota restante. Omitiendo S5AK08
Rampazo sin flota restante. Omitiendo S3AX01
Rampazo sin flota res

C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  'MES': int(horizonte.loc[horizonte['DIA']==day,'MES']),
C:\Users\wilma\AppData\Local\Temp\ipykernel_29000\1547251297.py:114: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) in